In [1]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [2]:
gmsh.open("twist2D.geo")

Info    : Reading 'twist2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000159254s, CPU 0.000154s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 5.0383e-05s, CPU 4e-05s)
Info    : 6 nodes 12 elements
Info    : Done reading 'twist2D.geo'


In [3]:
mat = FEM.material("body", E=2.e5, ν=0.3)
problem = FEM.Problem([mat], type="AxiSymmetric", bandwidth="none")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 3)


LowLevelFEM.Problem("twist2D", "AxiSymmetric", 2, [("body", 200000.0, 0.3, 7.85e-9)], 1.0, 6)

In [4]:
#gmsh.fltk.run()

In [5]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [6]:
K = FEM.stiffnessMatrix(problem)

12×12 SparseArrays.SparseMatrixCSC{Float64, Int64} with 112 stored entries:
  2.03398e7   6.24291e6  -3.62491e6  …  -1.00692e7  -2.81938e6  -1.0472e7
  6.24291e6   8.56657e6  -3.22215e6     -5.11206e6   3.22215e6  -5.08108e6
 -3.62491e6  -3.22215e6   1.22844e7      2.21523e6  -3.02076e6   3.8263e6
  7.04845e6   1.25478e6  -2.81938e6     -8.11733e6   5.23599e6  -5.11206e6
   ⋅           ⋅           ⋅            -9.46505e6  -1.38955e7  -6.24291e6
   ⋅           ⋅           ⋅         …  -8.11733e6  -1.00692e7  -5.11206e6
   ⋅           ⋅           ⋅             5.23599e6  -2.81938e6   3.22215e6
   ⋅           ⋅           ⋅            -5.11206e6  -1.0472e7   -5.08108e6
 -1.38955e7  -6.24291e6  -5.63876e6      1.08747e7  -8.45813e6   6.04152e6
 -1.00692e7  -5.11206e6   2.21523e6      2.67376e7   1.2083e6    4.92617e6
 -2.81938e6   3.22215e6  -3.02076e6  …   1.2083e6    3.10132e7   3.62491e6
 -1.0472e7   -5.08108e6   3.8263e6       4.92617e6   3.62491e6   2.19664e7

In [7]:
supp = FEM.displacementConstraint("supp", uy=0)
load = FEM.load("load", fy=1)

("load", 0, 1, 0)

In [8]:
f = FEM.loadVector(problem, [load])

"nodeTags = UInt64[0x0000000000000001, 0x0000000000000002, 0x0000000000000005, 0x0000000000000002, 0x0000000000000003, 0x0000000000000003, 0x0000000000000004, 0x0000000000000006, 0x0000000000000004, 0x0000000000000001]"

"ncoord = [10.0, 0.0, 0.0, 20.0, 0.0, 0.0, 20.0, 9.999999999984798, 0.0, 20.0, 0.0, 0.0, 20.0, 20.0, 0.0, 20.0, 20.0, 0.0, 10.0, 20.0, 0.0, 10.0, 9.999999999999993, 0.0, 10.0, 20.0, 0.0, 10.0, 0.0, 0.0]"

"ncoord2 = [10.0, 0.0, 0.0, 20.0, 0.0, 0.0, 20.0, 20.0, 0.0, 10.0, 20.0, 0.0, 20.0, 9.999999999984798, 0.0, 10.0, 9.999999999999993, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"

12-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
 523.5987755982989
   0.0
 418.8790204786392
   0.0
   0.0
   0.0
   0.0

In [9]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

In [10]:
q = FEM.solveDisplacement(K, f)

12-element Vector{Float64}:
  3.952116004866008e-5
  0.0
 -1.9158952745028902e-5
  0.0
 -9.198732473001942e-5
  0.00019710999072716662
  6.327270951685674e-5
  0.00016077181045589942
 -3.895783079860453e-5
  6.482272757632681e-5
  6.49357465156712e-5
  5.527599633440585e-5

In [11]:
S = FEM.solveStress(problem, q)

ErrorException: stiffnessMatrixSolid: dimension is 2, problem type is AxiSymmetric.

In [12]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)

In [12]:
u = FEM.showDoFResults(problem, q, "uvec", name="uvec", visible=false)
ux = FEM.showDoFResults(problem, q, "ux", name="ux", visible=false)
uy = FEM.showDoFResults(problem, q, "uy", name="uy", visible=false)
uz = FEM.showDoFResults(problem, q, "uz", name="uz", visible=false)
s = FEM.showStressResults(problem, S, "s", name="σ", visible=true, smooth=true)
sx = FEM.showStressResults(problem, S, "sx", name="σx", visible=false, smooth=true)
sy = FEM.showStressResults(problem, S, "sy", name="σy", visible=false, smooth=true)
sz = FEM.showStressResults(problem, S, "sz", name="σz", visible=false, smooth=true)
sxy = FEM.showStressResults(problem, S, "sxy", name="τxy", visible=false, smooth=true)
syz = FEM.showStressResults(problem, S, "syz", name="τyz", visible=false, smooth=true)
szx = FEM.showStressResults(problem, S, "szx", name="τzx", visible=false, smooth=true)

UndefVarError: UndefVarError: `S` not defined

In [14]:
FEM.plotOnPath(problem, "path", sx, 100, name="σx", visible=false);
FEM.plotOnPath(problem, "path", sxy, 100, name="τxy", visible=false);
FEM.plotOnPath(problem, "path", ux, 100, name="ux", visible=false);

In [15]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [13]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()